In [ ]:
import requests
import pandas as pd
import time
from dotenv import load_dotenv
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 1. Abrir base de datos original.

In [ ]:
df = pd.read_excel("bd.xlsx")
df_2 = df[["Primer_Nombre", "Apellido_Paterno"]]
print(df_2.shape)
names = df_2.to_dict("records")  # Convertimos el dataframe a una lista de diccionarios

# 2. Carga de variable de entornos 

In [ ]:
load_dotenv()
API_KEY = os.getenv('API_KEY')
INST_TOKEN = os.getenv('INST_TOKEN')

# 3. Funciones para descargar datos de Scopus 

## 3.1 Función para obtener los datos del autor según nombre y apellido.

In [ ]:
def get_author_id(author_name, author_lastname):
    URL = f"https://api.elsevier.com/content/search/author?query=authlast({author_lastname})%20and%20authfirst({author_name})&apiKey={API_KEY}"
    # Debe haber un delay de 3 segundos entre cada request para no exceder el límite de requests por segundo.
    time.sleep(3)
    response = requests.get(
        URL,
        headers={
            "Accept": "application/json",
            "X-ELS-APIKey": API_KEY,
            "X-ELS-Insttoken": INST_TOKEN,
        },
    )
    data = response.json()
    # Si la respuesta contiene un "entry" significa que se encontró un autor con ese nombre.
    if "search-results" in data:
        # Retornamos todos los autores encontrados
        return [
            {
                "entry": entry,
                "author_name": author_name,
                "author_lastname": author_lastname,
            }
            for entry in data["search-results"]["entry"]
        ]
    # Si no se encontró un autor con ese nombre, retornamos un diccionario con el error y los datos del autor. 
    return [
        {
            "Error": response,
            "entry": None,
            "author_name": author_name,
            "author_lastname": author_lastname,
        }
    ]

# 3.2 Función para realizar request de forma paralela

In [ ]:
def fetch_all_authors(names):
    # Usamos un ThreadPoolExecutor para hacer las requests en paralelo.
    results = []
    # Usamos un máximo de 2 workers para no exceder el límite de requests por segundo.
    with ThreadPoolExecutor(max_workers=2) as executor:
        # Usamos la función submit para enviar las requests al executor.
        futures = [executor.submit(get_author_id,  # Enviamos la función que queremos ejecutar.
                                   # Enviamos el nombre y apellido del autor.
                                   name['Primer_Nombre'], 
                                   name['Apellido_Paterno']) 
                   for name in names] # Iteramos sobre la lista de nombres
        # Iteramos sobre los resultados de las futures y los guardamos en una lista.
        for future in futures:
            results.extend(future.result()) # Usamos el método result para obtener el resultado de la future.
    return results

In [ ]:
# Dividimos la lista de nombres total en x partes para hacer requests en paralelo. Esto es recomendable, en caso de error no se pierden todas las requests.
X = 1000
all_results = fetch_all_authors(names[X-X:X])
all_results

# 4. Exportar datos obtenidos

In [ ]:
results_list = []

In [ ]:
for result in all_results:
    try:
        # Si el resultado contiene un error, lo imprimimos y continuamos con el siguiente resultado.
        if 'error' in result:
            print(f"Error in result: {result['error']}")
            continue
        # Extraer los datos del resultado.
        entry = result.get("entry", {})
        author_name = result.get("author_name", "N/A") # Si no se encuentra el nombre, se asigna "N/A".
        author_lastname = result.get("author_lastname", "N/A") # Si no se encuentra el apellido, se asigna "N/A".
        author_id = entry.get("dc:identifier", "N/A").split(":")[-1] # Extraer el ID del autor.
        preferred_name = entry.get("preferred-name", {}) # Extraer el nombre preferido del autor.
        surname = preferred_name.get("surname", "N/A") # Extraer el apellido del autor.
        given_name = preferred_name.get("given-name", "N/A") # Extraer el nombre del autor.
        affiliation = entry.get("affiliation-current", {}) # Extraer la afiliación actual del autor.
        affiliation_name = affiliation.get("affiliation-name", "N/A") # Extraer el nombre de la afiliación.
        affiliation_city = affiliation.get("affiliation-city", "N/A") # Extraer la ciudad de la afiliación.
        affiliation_country = affiliation.get("affiliation-country", "N/A") # Extraer el país de la afiliación.
        document_count = entry.get("document-count", "N/A") # Extraer el número de documentos del autor (OPCIONAL).
        
        # Si el campo "subject-area" es una lista, extraer los valores y unirlos en un string separado por comas. 
        # Esto no siempre funciona, ya que a veces el campo "subject-area" no es una lista. Por lo que aplicar una limpieza adicional si es necesario.
        if isinstance(entry.get("subject-area"), list):
            subject_areas = ", ".join([area.get("$", "N/A") if "$" in area else "N/A" for area in entry["subject-area"]])  # Extraer los valores de la lista.
        else:
            subject_areas = entry.get("subject-area", "N/A") # Si el campo no es una lista, extraer el valor directamente.
        
        # Agregar los resultados a la lista
        results_list.append({
            "Author ID": author_id, # Agregar el ID del autor.
            "Name": f"{given_name} {surname}", # Agregar el nombre completo del autor.
            "Affiliation": f"{affiliation_name}, {affiliation_city}, {affiliation_country}", # Agregar la afiliación del autor.
            "Document count": document_count, # Agregar el número de documentos del
            "Subject areas": subject_areas, # Agregar las áreas de investigación del autor.
            "Search Name": f"{author_name} {author_lastname}" # Agregar el nombre de búsqueda.
        })
    except KeyError as e:
        print(f"KeyError: Missing key {e} in result {result}") # Si falta una llave en el diccionario, imprimir el error.
    except Exception as e:
        print(f"Unexpected error: {e} in result {result}") # Si ocurre un error inesperado, imprimir el error.

In [ ]:
# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results_list)

# Guardar los resultados en un archivo Excel
df_results.to_excel("resultado_X_X.xlsx")